In [1]:
import pandas as pd

Cargamos la data correspondiente

In [2]:
df_reviews = pd.read_parquet("../Datasets/reviews_analisis_sentiment.parquet")
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")

In [3]:
df_reviews.head(2)

,user_id,user_url,reviews_item_id,reviews_recommend,reviews_date,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011-11-05,2011,2
1,js41637,http://steamcommunity.com/id/js41637,251610,True,2014-06-24,2014,2


In [4]:
df_games.head(2)

,publisher,genres,title,reviews_url,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0
0,Kotoshiro,Casual,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018.0


Seleccionamos las columnas necesarias para el dataframe games

In [5]:
df_games = df_games[["id", "developer", "year"]]

seleccionamos las columnas necesarias de reviews

In [6]:
df_reviews = df_reviews[["reviews_item_id", "reviews_recommend", "sentiment_analysis"]]

Unimos los dos dataframes

In [7]:
df_works_dev = pd.merge(df_games, df_reviews, left_on="id", right_on="reviews_item_id", how="inner")

In [8]:
df_works_dev

,id,developer,year,reviews_item_id,reviews_recommend,sentiment_analysis
0,70,Valve,1998.0,70,True,1
1,70,Valve,1998.0,70,True,1
2,70,Valve,1998.0,70,True,1
3,70,Valve,1998.0,70,True,2
4,70,Valve,1998.0,70,True,2
...,...,...,...,...,...,...
100224,80,Valve,2004.0,80,False,1
100225,80,Valve,2004.0,80,True,2
100226,80,Valve,2004.0,80,True,2
100227,80,Valve,2004.0,80,True,1


Creamos la funcion

In [15]:
def userWorkstDeveloper(año: int):
    df = df_works_dev[df_works_dev["year"]== año]
    #filtramos con comenntarios negativos
    games_recommend = df[(df["reviews_recommend"]== False) & (df["sentiment_analysis"]== 1)]
    
    developer_recommend = games_recommend["developer"].value_counts().reset_index()
    developer_recommend.columns = ["developer", "recommend_count"]
    
    # Ordenamos
    top_developer = developer_recommend.nlargest(3, "recommend_count")
    top_developer.sort_values("recommend_count", ascending=False)
    
    resultado = [{"Puesto {}: {}".format(i + 1, row['developer']): row['recommend_count']} for i, row in top_developer.iterrows()] # type: ignore
    
    return resultado

Probando la funcion

In [20]:
userWorkstDeveloper(2017)

[{'Puesto 1: Smartly Dressed Games': 170},
 {'Puesto 2: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC': 95},
 {'Puesto 3: Freejam': 78}]

Cargamos la data

In [21]:
df_works_dev.to_parquet("../Funciones/data/userWorstDeveloper.parquet")